# Leaf shape analysis using Generalized Procrustes Analysis 

The following is a `jupyter notebook` ([Kluyver et al. 2016](https://ebooks.iospress.nl/doi/10.3233/978-1-61499-649-1-87)) tutorial written using the `python` coding language. Text written in `markdown cells` is used to explain code presented and executed in `coding cells`. This tutorial assumes a working knowledge of `python` and the ability to use `jupyter notebooks`.  

If you are new to `python` or do not know how to use `jupyter notebooks`, we recommend that you familiarize yourself with them through a tutorial. For the context of plant biology and leaf shape presented here, we recommend `Plants&Python` ([VanBuren et al., 2022](https://doi.org/10.1093/plcell/koac187)), accessible using this [link](https://plantsandpython.github.io/PlantsAndPython). There you will find instructions for downloading and installing [Anaconda](https://docs.anaconda.com/anaconda/install/) and how to get going with `jupyter notebooks` and `python`.m

<a class="anchor" id="page_top"></a>

## Sections
1. [Loading modules and functions](#Loading_modules_and_functions)
2. [Reading in data](#Reading_in_data)

    a. [*Capsella bursa-pastoris*](#capsella)
    
    b. [*Arabidopsis thaliana*](#arabidopsis)
    
    c. [*Quercus spp*](#oak)
    
    d. [*Cannabis sativa*](#cannabis)
    
    e. [*Beta vulgaris*](#sugar_beets)
    
    f. [*Lobelia spp*](#lobelia)
    
    g. [*Erythroxylum coca*](#coca)
    
    h. [*Malus spp*](#apple)
    
    
3. [Plotting pseudo-landmarks](#pseudo_landmarks)
4. [Section 4: combine datasets](#combine_datasets)
5. [Section 5: Procruestes Analysis and PCA](#pca)
6. [Section 6: Save pseudo-landmarked and Procrustes aligned leaves for future use](#save_data)
7. [Section 7: Analyze leaf dimensions](#leaf_measurements)
8. [Section 8: Explore data - Pairplot of all measured traits](#pair_plot)
9. [Color Palette](#color_palette)
10. [Section 9: Plotting PCA morphospace](#morphospace)
11. [Continuous color - custom](#custom_color)
12. [Section 10: Plotting real leaves](#plot_leaves)
13. [Section 11: Split datasets for further PCA morphospace plotting](#split_dataset)
14. [Section 12: Linear Discriminant Analysis](#LDA)
15. [Section 14: Plotting the mean leaf for the entire dataset and each species dataset](#mean_leaf)

<a class="anchor" id="Loading_modules_and_functions"></a>

### Section 1: Loading modules and functions

In [ ]:
#######################
### LOAD IN MODULES ###
#######################

import cv2 # to install on mac: pip install opencv-python
from scipy.interpolate import interp1d # for interpolating points
from sklearn.decomposition import PCA # for principal component analysis
from scipy.spatial import procrustes # for Procrustes analysis
from scipy.spatial import ConvexHull # for convex hull
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # for LDA
from sklearn.metrics import confusion_matrix # for confusion matrix
import os
from os import listdir # for retrieving files from directory
from os.path import isfile, join # for retrieving files from directory
import matplotlib.pyplot as plt # for plotting
%matplotlib inline
import numpy as np # for using arrays
import math # for mathematical operations
import pandas as pd # for using pandas dataframes
import seaborn as sns # for plotting in seaborna
import csv
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [ ]:
#################
### FUNCTIONS ###
#################

def angle_between(p1, p2, p3):
    """
    define a function to find the angle between 3 points anti-clockwise in degrees, p2 being the vertex
    inputs: three angle points, as tuples
    output: angle in degrees
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    deg1 = (360 + math.degrees(math.atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + math.degrees(math.atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

def rotate_points(xvals, yvals, degrees):
    """"
    define a function to rotate 2D x and y coordinate points around the origin
    inputs: x and y vals (can take pandas dataframe columns) and the degrees (positive, anticlockwise) to rotate
    outputs: rotated and y vals
    """
    angle_to_move = 90-degrees
    rads = np.deg2rad(angle_to_move)
    
    new_xvals = xvals*np.cos(rads)-yvals*np.sin(rads)
    new_yvals = xvals*np.sin(rads)+yvals*np.cos(rads)
    
    return new_xvals, new_yvals

def interpolation(x, y, number): 
    """
    define a function to return equally spaced, interpolated points for a given polyline
    inputs: arrays of x and y values for a polyline, number of points to interpolate
    ouputs: interpolated points along the polyline, inclusive of start and end points
    """
    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular

def euclid_dist(x1, y1, x2, y2):
    """
    define a function to return the euclidean distance between two points
    inputs: x and y values of the two points
    output: the eulidean distance
    """
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def poly_area(x,y):
    """
    define a function to calculate the area of a polygon using the shoelace algorithm
    inputs: separate numpy arrays of x and y coordinate values
    outputs: the area of the polygon
    """
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def gpa_mean(leaf_arr, landmark_num, dim_num):
    
    """
    define a function that given an array of landmark data returns the Generalized Procrustes Analysis mean
    inputs: a 3 dimensional array of samples by landmarks by coordinate values, number of landmarks, number of dimensions
    output: an array of the Generalized Procrustes Analysis mean shape
    
    """

    ref_ind = 0 # select arbitrary reference index to calculate procrustes distances to
    ref_shape = leaf_arr[ref_ind, :, :] # select the reference shape

    mean_diff = 10**(-30) # set a distance between means to stop the algorithm

    old_mean = ref_shape # for the first comparison between means, set old_mean to an arbitrary reference shape

    d = 1000000 # set d initially arbitraily high

    while d > mean_diff: # set boolean criterion for Procrustes distance between mean to stop calculations

        arr = np.zeros( ((len(leaf_arr)),landmark_num,dim_num) ) # empty 3D array: # samples, landmarks, coord vals

        for i in range(len(leaf_arr)): # for each leaf shape 

            s1, s2, distance = procrustes(old_mean, leaf_arr[i]) # calculate procrustes adjusted shape to ref for current leaf
            arr[i] = s2 # store procrustes adjusted shape to array

        new_mean = np.mean(arr, axis=(0)) # calculate mean of all shapes adjusted to reference

        s1, s2, d = procrustes(old_mean, new_mean) # calculate procrustes distance of new mean to old mean

        old_mean = new_mean # set the old_mean to the new_mea before beginning another iteration

    return new_mean

def Circularity(shape_arr):
    
    lines = np.hstack([shape_arr,np.roll(shape_arr,-1,axis=0)])
    area = 0.5*abs(sum(x1*y2-x2*y1 for x1,y1,x2,y2 in lines))
    
    distance = np.cumsum(np.sqrt( np.ediff1d(shape_arr[:,0], to_begin=0)**2 + np.ediff1d(shape_arr[:,1], to_begin=0)**2 ))
    perimeter = distance[-1]
    
    circularity = (4*math.pi*area)/perimeter**2
    
    return circularity


[Back to top](#page_top)

<a id='#Reading in data'></a>

### Section 2: Reading in Data  <a class="anchor" id="Reading_in_data"></a>

If your image data includes only outlines (xy coordinate files) follow subsection 1. If your image data includes only black leaf images printed on white backgound, follow subsection 2. 

##### Subsection 1: Reading in leaf outline data

In [ ]:
os.chdir('/Volumes/TOSHIBA/collab_v3/')

##### Subsection 2: Reading in binary leaf image data

Steps
1. Change the directory where your files are located. This should be one folder that includes another folder of all images you want to include in your data set and a csv file.
2. Import your metadata csv as a pandas dataframe named mdata. The csv file should include these headings: file - the file name for each image, dataset - researcher defined, px_cm - pixels per centimeter, base_x and base_y - the x and y coordinates for the base of each leaf, tip_x and tip_y - the x and y coordinates for the tip of each leaf.
3. Upload your images into the "data_dir" data directory using the file path for the folder with all of your images files for your dataset. 
4. Print the mdata dataframe and image file names to check for any errors. Most errors occur at this step.

<a class="anchor" id="capsella"></a>

### *Capsella bursa-pastoris*

In [ ]:
capsella_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/capsella/capsella.csv") # read in csv

capsella_mdata.head() # head data to check

In [ ]:
len(capsella_mdata)

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/capsella/capsella_images/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

<a class="anchor" id="pseudo_landmarks"></a>

### Section 3: Place equidistant pseudo-landmarks

- Read in image in grayscale
- Select the contour of the largest object (the leaf)
- Interpolate with a high resolution of pseudo-landmarks
- Find the base and tip index point on the high resolution contour
- Reset the base index to zero
- Interpolate each side with desired number of equidistant pseudo-landmarks
- Rotate leaves and scale to centimeters
- Save pseudo-landmarks scaled to centimeters in an array

PARAMETERS AND INDEXING:
- `high_res_pts` is an arbitrarily high number of points to initially interpolate
- `res` is the desired number of points to interpolate on each side of the leaf
- The total number of pseudo-landmarks will be `2*res - 1`
- The base index will be `0`
- The tip index will be `res-1`
- The returned leaves in `cm_arr` are scaled in size to centimeters

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 1000 

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
capsella_cm_arr = np.zeros((len(capsella_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(capsella_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = capsella_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(sorted_x_conts[0], 
                                           sorted_y_conts[0], high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((capsella_mdata["base_x"][lf], capsella_mdata["base_y"][lf]))
    tip_pt = np.array((capsella_mdata["tip_x"][lf], capsella_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))

    # scale leaf into cm
    cm_lf = rot_pts/(capsella_mdata["px_cm"][lf])
    
    # store the leaf scaled into cm into the cm_arr
    capsella_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("capsella_bursa_pastoris_cm_arr", capsella_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="arabidopsis"></a>

### *Arabidopsis thaliana*

In [ ]:
arabidopsis_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/arabidopsis/arabidopsis.csv") # read in csv

arabidopsis_mdata.head() # head data to check

In [ ]:
len(arabidopsis_mdata)

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/arabidopsis/arabidopsis_images/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 1000 

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
arabidopsis_cm_arr = np.zeros((len(arabidopsis_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(arabidopsis_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = arabidopsis_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(sorted_x_conts[0], 
                                           sorted_y_conts[0], high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((arabidopsis_mdata["base_x"][lf], arabidopsis_mdata["base_y"][lf]))
    tip_pt = np.array((arabidopsis_mdata["tip_x"][lf], arabidopsis_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))

    # scale leaf into cm
    cm_lf = rot_pts/(arabidopsis_mdata["px_cm"][lf])
    
    # store the leaf scaled into cm into the cm_arr
    arabidopsis_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("arabidopsis_thaliana_cm_arr", arabidopsis_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="oak"></a>

### *Quercus spp*

In [ ]:
oak_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/oak/oak.csv") # read in csv

oak_mdata.head() # head data to check

In [ ]:
len(oak_mdata)

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/oak/claire_oak_reprint_111224/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 1000 

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
oak_cm_arr = np.zeros((len(oak_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(oak_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = oak_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(sorted_x_conts[0], 
                                           sorted_y_conts[0], high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((oak_mdata["base_x"][lf], oak_mdata["base_y"][lf]))
    tip_pt = np.array((oak_mdata["tip_x"][lf], oak_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))

    # scale leaf into cm
    cm_lf = rot_pts/(oak_mdata["px_cm"][lf])
    
    # store the leaf scaled into cm into the cm_arr
    oak_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("Quercus_spp_cm_arr", oak_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="cannabis"></a>

### *Cannaibas sativa*

In [ ]:
cannibas_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/cannibas/cannibas.csv") # read in csv

cannibas_mdata.head() # head data to check

In [ ]:
len(cannibas_mdata)

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/cannibas/cannibas_images/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 1000 

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
cannibas_cm_arr = np.zeros((len(cannibas_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(cannibas_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = cannibas_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(sorted_x_conts[0], 
                                           sorted_y_conts[0], high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((cannibas_mdata["base_x"][lf], cannibas_mdata["base_y"][lf]))
    tip_pt = np.array((cannibas_mdata["tip_x"][lf], cannibas_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))

    # scale leaf into cm
    cm_lf = rot_pts/(cannibas_mdata["px_cm"][lf])
    
    # store the leaf scaled into cm into the cm_arr
    cannibas_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("cannibas_sativa_cm_arr", cannibas_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="sugar_beets"></a>

### *Beta vulgaris*

In [ ]:
########################
### READ IN METADATA ###
########################

sugar_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/sugar_beets/sugar_beets.csv") # read in csv

sugar_mdata.head() # head data to check

In [ ]:
len(sugar_mdata)

In [ ]:
data_dir = "/Volumes/TOSHIBA/collab_v3/sugar_beets/LEAF_SCAN_BINARY/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 10000

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
sugar_cm_arr = np.zeros((len(sugar_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(sugar_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = sugar_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(np.array(sorted_x_conts[0], dtype=np.float32), 
                                           np.array(sorted_y_conts[0], dtype=np.float32), high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((sugar_mdata["base_x"][lf], sugar_mdata["base_y"][lf]))
    tip_pt = np.array((sugar_mdata["tip_x"][lf], sugar_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))
    
    # calculate leaf area in pixels^2
    lf_area_px2 = poly_area(rot_pts[:,0], rot_pts[:,1])
    
    # get px_cm
    px_cm = sugar_mdata["px_cm"][lf]

    # scale leaf into cm
    cm_lf = rot_pts/(px_cm)
    
    # store the leaf scaled into cm into the cm_arr
    sugar_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("beta_vulgaris_cm_arr", sugar_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="lobelia"></a>

### *Lobelia spp*

In [ ]:
lobelia_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/lobelia/lobelia.csv") # read in csv

lobelia_mdata.head()

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/lobelia/binary_leaves/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 10000

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
lobeliacm_arr = np.zeros((len(lobelia_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(lobelia_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = lobelia_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(np.array(sorted_x_conts[0], dtype=np.float32), 
                                           np.array(sorted_y_conts[0], dtype=np.float32), high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((lobelia_mdata["base_x"][lf], lobelia_mdata["base_y"][lf]))
    tip_pt = np.array((lobelia_mdata["tip_x"][lf], lobelia_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))
    
    # calculate leaf area in pixels^2
    lf_area_px2 = poly_area(rot_pts[:,0], rot_pts[:,1])
    
    # get px_cm
    px_cm = lobelia_mdata["px_cm"][lf]

    # scale leaf into cm
    cm_lf = rot_pts/(px_cm)
    
    # store the leaf scaled into cm into the cm_arr
    lobeliacm_arr[lf,:,:] = cm_lf
    

In [ ]:
np.save("lobelia_spp_cm_arr", lobeliacm_arr)

[Back to top](#page_top)

<a class="anchor" id="coca"></a>

### *Erythroxylum coca*

In [ ]:
cocoa_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/cocoa/cocoa.csv") # read in csv

cocoa_mdata.head()

In [ ]:
data_dir = "/Volumes/TOSHIBA/collab_v3/cocoa/cultivated_images/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 10000

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
cocoa_cm_arr = np.zeros((len(cocoa_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(cocoa_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = cocoa_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(np.array(sorted_x_conts[0], dtype=np.float32), 
                                           np.array(sorted_y_conts[0], dtype=np.float32), high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((cocoa_mdata["base_x"][lf], cocoa_mdata["base_y"][lf]))
    tip_pt = np.array((cocoa_mdata["tip_x"][lf], cocoa_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))
    
    # calculate leaf area in pixels^2
    lf_area_px2 = poly_area(rot_pts[:,0], rot_pts[:,1])
    
    # get px_cm
    px_cm = cocoa_mdata["px_cm"][lf]

    # scale leaf into cm
    cm_lf = rot_pts/(px_cm)
    
    # store the leaf scaled into cm into the cm_arr
    cocoa_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("Erythroxylum_coca_cm_arr", cocoa_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="apple"></a>

### *Malus spp*

In [ ]:
apple_mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/apple/apple.csv") # read in csv

apple_mdata.head() # head data to check

In [ ]:
#######################################
### MAKE A LIST OF IMAGE FILE NAMES ###
#######################################

data_dir = "/Volumes/TOSHIBA/collab_v3/apple/apple_images/" # set data directory

file_names = [f for f in listdir(data_dir) if isfile(join(data_dir, f))] # create a list of file names

#file_names.remove('.DS_Store') # remove .DS_Store file

file_names.sort() # sort the list of file names

file_names # check list of file names

In [ ]:
######################
### SET PARAMETERS ###
######################

# the number of equidistant points to create
# an initial high resolution outline of the leaf
high_res_pts = 10000

# the ultimate number of equidistant points on each side of the leaf
# (-1 for the tip)
# the leaf will have res*2-1 pseudo-landmarks
#################
#################
#################
res = 50 ########
#################
#################
#################

# an array to store pseudo-landmarks
apple_cm_arr = np.zeros((len(apple_mdata),(res*2)-1,2))

# for each leaf . . .
for lf in range(len(apple_mdata)):

    ###############################
    ### READ IN GRAYSCALE IMAGE ###
    ###############################

    curr_image = apple_mdata["file"][lf] # select the current image
    print(lf, curr_image) # print each leaf in case there are problems later

    # read in image
    # convert to grayscale
    # invert the binary
    img = cv2.bitwise_not(cv2.cvtColor(cv2.imread(data_dir + curr_image),cv2.COLOR_BGR2GRAY))

    # find contours of binary objects
    contours, hierarchy = cv2.findContours(img,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ##############################
    ### SELECT LARGEST CONTOUR ###
    ##############################

    # ideally there is only one leaf in the image
    # in the case there are smaller objects
    # this code selects the largest object (the leaf)
    # if there is one and only one object in the image
    # then the following code is not necessary

    x_conts = [] # list of lists of contour x vals
    y_conts = [] # list of lists of contour y vals
    areas_conts = [] # list of bounding box areas of contours
    for c in contours: # for each contour
        x_vals = [] # store x vals for current contour 
        y_vals = [] # store y vals for current contour
        for i in range(len(c)): # for each point in current contour
            x_vals.append(c[i][0][0]) # isolate x val
            y_vals.append(c[i][0][1]) # isolate y val
        area = (max(x_vals) - min(x_vals))*(max(y_vals) - min(y_vals)) # calculate bounding box area of contour
        x_conts.append(x_vals) # append the current contour x vals
        y_conts.append(y_vals) # append the current contour y vals
        areas_conts.append(area) # append the current contour bounding box areas

    area_inds = np.flip(np.argsort(areas_conts)) # get indices to sort contours by area
    sorted_x_conts = np.array(x_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, x vals
    sorted_y_conts = np.array(y_conts, dtype=object)[area_inds][0:] # areas sorted largest to smallest, y vals

    ################################################
    ### INTERPOLATE HIGH RES NUMBER OF LANDMARKS ###
    ################################################

    # convert the leaf to high resolution number of landmarks
    # using high_res_pt value
    # need to convert arrays of pixel int to floats first
    high_res_x, high_res_y = interpolation(np.array(sorted_x_conts[0], dtype=np.float32), 
                                           np.array(sorted_y_conts[0], dtype=np.float32), high_res_pts)

    ###############################
    ### FIND BASE AND TIP INDEX ###
    ###############################

    # get the base and tip landmark point values
    base_pt = np.array((apple_mdata["base_x"][lf], apple_mdata["base_y"][lf]))
    tip_pt = np.array((apple_mdata["tip_x"][lf], apple_mdata["tip_y"][lf]))

    base_dists = [] # store distance of each high res point to base
    tip_dists = [] # store distance of each high res point to tip

    for pt in range(len(high_res_x)): # for each of the high resolution points

        # euclidean distance of the current point from the base and tip landmark
        ed_base = euclid_dist(base_pt[0], base_pt[1], high_res_x[pt], high_res_y[pt])
        ed_tip = euclid_dist(tip_pt[0], tip_pt[1], high_res_x[pt], high_res_y[pt])

        # store distance of current point from base/tip
        base_dists.append(ed_base)
        tip_dists.append(ed_tip)

    # get index of base and tip points
    base_ind = np.argmin(base_dists)
    tip_ind = np.argmin(tip_dists)

    ################################
    ### RESET BASE INDEX TO ZERO ###
    ################################

    # reset base index position to zero
    high_res_x = np.concatenate((high_res_x[base_ind:],high_res_x[:base_ind]))
    high_res_y = np.concatenate((high_res_y[base_ind:],high_res_y[:base_ind]))

    # recalculate indices with new indexing
    tip_ind = tip_ind-base_ind # note: negative index if tip_ind<base_ind
    base_ind = base_ind-base_ind

    # create single array for leaf coordinates
    lf_contour = np.column_stack((high_res_x, high_res_y))

    ##############################################################
    ### INTERPOLATE EACH SIDE WITH DESIRED NUMBER OF LANDMARKS ###
    ##############################################################

    # interpolate at desired resolution the left and right sides of the leaf
    left_inter_x, left_inter_y = interpolation(lf_contour[base_ind:tip_ind+1,0],lf_contour[base_ind:tip_ind+1,1],res)
    right_inter_x, right_inter_y = interpolation(lf_contour[tip_ind:,0],lf_contour[tip_ind:,1],res)

    # the start of the right side and end of the left side
    # both contain the tip landmark
    # delete the last point on the left side
    left_inter_x = np.delete(left_inter_x, -1)
    left_inter_y = np.delete(left_inter_y, -1)

    # BASE OF LEAF IS INDEX 0
    # TIP INDEX IS RES-1 IF BOTH LEFT & RIGHT POINTS
    # TOTAL PSEUDOLANDMARKS IS 2*RES-1
    lf_pts_left = np.column_stack((left_inter_x, left_inter_y))
    lf_pts_right = np.column_stack((right_inter_x, right_inter_y))
    lf_pts = np.row_stack((lf_pts_left, lf_pts_right))

    ##########################################################
    ### ROTATE LEAVES UPWARD AND SCALE SIZE TO CENTIMETERS ###
    ##########################################################

    tip_point = lf_pts[res-1,:] # get tip point
    base_point = lf_pts[0,:] # get base point

    # calculate angle between tip. base, and an arbitrary reference
    ang = angle_between(tip_point, base_point, (base_point[0]+1,base_point[1]) )

    # rotate points upwards
    rot_x, rot_y = rotate_points(lf_pts[:,0], lf_pts[:,1], ang) 
    rot_pts = np.column_stack((rot_x, rot_y))
    
    # calculate leaf area in pixels^2
    lf_area_px2 = poly_area(rot_pts[:,0], rot_pts[:,1])
    
    # get px_cm
    px_cm = apple_mdata["px_cm"][lf]

    # scale leaf into cm
    cm_lf = rot_pts/(px_cm)
    
    # store the leaf scaled into cm into the cm_arr
    apple_cm_arr[lf,:,:] = cm_lf

In [ ]:
np.save("malus_spp_cm_arr", apple_cm_arr)

[Back to top](#page_top)

<a class="anchor" id="combine_datasets"></a>

## Section 4: Combine datasets

In [ ]:
# stack domesticated and wild arrays into one new array

cm_arr = np.row_stack((capsella_cm_arr, arabidopsis_cm_arr, oak_cm_arr, lobeliacm_arr, 
                       cannibas_cm_arr, cocoa_cm_arr, apple_cm_arr, sugar_cm_arr))


In [ ]:
capsella_cm_arr = np.load("capsella_cm_arr.npy")
arabidopsis_cm_arr = np.load("arabidopsis_cm_arr.npy")
oak_cm_arr = np.load("oak_cm_arr.npy")
lobeliacm_arr = np.load("lobeliacm_arr.npy") 
cannibas_cm_arr = np.load("cannibas_cm_arr.npy")
cocoa_cm_arr = np.load("cocoa_cm_arr.npy")
apple_cm_arr = np.load("apple_cm_arr.npy")
sugar_cm_arr = np.load("sugar_cm_arr.npy")

In [ ]:
# read in each dataset

capsella_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/capsella/capsella.csv") 
arabidopsis_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/arabidopsis/arabidopsis.csv")
oak_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/oak/oak.csv")
cannibas_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/cannibas/cannibas.csv")
lobelia_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/lobelia/lobelia.csv")
sugar_beets_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/sugar_beets/sugar_beets.csv")
cocoa_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/cocoa/cocoa.csv")
apple_data = pd.read_csv("/Volumes/TOSHIBA/collab_v3/apple/apple.csv")

# stack domesticated and wild genotypes into one new dataframe

mdata = pd.concat([capsella_data, arabidopsis_data, oak_data, lobelia_data, cannibas_data, 
                        cocoa_data, apple_data, sugar_beets_data], ignore_index=True)

In [ ]:
os.chdir('/Volumes/TOSHIBA/collab_v3/')

In [ ]:
mdata.to_csv('mdata_v1.csv', header=True)

In [ ]:
mdata = pd.read_csv("/Volumes/TOSHIBA/collab_v3/mdata_v1.csv") 
mdata.head()

[Back to top](#page_top)

<a class="anchor" id="pca"></a>

## Section 5: Procrustes Analysis and PCA

Perform a Procrustes analysis to translate, scale, and rotate leaf shapes

- Select number of pseudo-landmarks and dimensions
- Calculate the GPA mean leaf shape using the `gpa_mean` function
- Align all leaves to the GPA mean
- Store Procrustes super-imposed leaves in an array, `proc_arr`
- Calculate a PCA for all possible axes and their variance (the number of leaves)
- Calculate a PCA for just the axes needed for reconstruction of eigenleaves for morphospace (probably 2)

In [ ]:
res = 50

In [ ]:
landmark_num = (res*2)-1 # select number of landmarks
dim_num = 2 # select number of coordinate value dimensions

##########################
### CALCULATE GPA MEAN ###
##########################

mean_shape = gpa_mean(cm_arr, landmark_num, dim_num)

################################
### ALIGN LEAVES TO GPA MEAN ###
################################

# array to store Procrustes aligned shapes
proc_arr = np.zeros(np.shape(cm_arr)) 

for i in range(len(cm_arr)):
    s1, s2, distance = procrustes(mean_shape, cm_arr[i,:,:]) # calculate procrustes adjusted shape to ref for current leaf
    proc_arr[i] = s2 # store procrustes adjusted shape to array
    
#################################################
### FIRST, CALCULATE PERCENT VARIANCE ALL PCs ###
#################################################

######
PC_NUMBER = np.shape(proc_arr)[1] # PC number = number of leaves
#######

# use the reshape function to flatten to 2D
flat_arr = proc_arr.reshape(np.shape(proc_arr)[0], 
                                 np.shape(proc_arr)[1]*np.shape(proc_arr)[2]) 

pca_all = PCA(n_components=2) 
PCs_all = pca_all.fit_transform(flat_arr) # fit a PCA for all data

# print out explained variance for each PC
print("PC: " + "var, " + "overall ") 
for i in range(len(pca_all.explained_variance_ratio_)):
    print("PC" + str(i+1) + ": " + str(round(pca_all.explained_variance_ratio_[i]*100,1)) + 
          "%, " + str(round(pca_all.explained_variance_ratio_.cumsum()[i]*100,1)) + "%"  )

#################################################
### NEXT, CALCULATE THE DESIRED NUMBER OF PCs ###
#################################################

######
PC_NUMBER = 2 # PC number = 2, for limiting to 2 axes for morphospace reconstruction
#######

pca = PCA(n_components=PC_NUMBER) 
PCs = pca.fit_transform(flat_arr) # fit a PCA for only desired PCs

# print out explained variance for each PC
print("PC: " + "var, " + "overall ") 
for i in range(len(pca.explained_variance_ratio_)):
    print("PC" + str(i+1) + ": " + str(round(pca.explained_variance_ratio_[i]*100,1)) + 
          "%, " + str(round(pca.explained_variance_ratio_.cumsum()[i]*100,1)) + "%"  )
    
# add PCs to dataframe for plotting
mdata["PC1"] = PCs[:,0]
mdata["PC2"] = PCs[:,1]

[Back to top](#page_top)

<a class="anchor" id="save_data"></a>

## Section 6: Save pseudo-landmarked and Procrustes aligned leaves for future use

In [ ]:
# Save rotated leaves with pseudo-landmarks
np.save('cm_arr', cm_arr)
#save procurestes aligned leaves
np.save('proc_arr', proc_arr)
#save flattened procurestes aligned leaves
np.save('flat_arr', flat_arr)

In [ ]:
import os 
file_names = mdata['file']
output_directory = './procurestes_files/'

for name, row in zip(file_names, proc_arr):
    filename = os.path.join(output_directory, f"{name}.txt")
    np.savetxt(filename, row, fmt='%.6f') 

When rerunning this jupyter notebook, it is faster to load the saved arrays that contain the pseudo-landmarks than to reprocess each species dataset.

In [ ]:
cm_arr = np.load("cm_arr.npy")
proc_arr = np.load("proc_arr.npy")
flat_arr = np.load("flat_arr.npy")

[Back to top](#page_top)

<a class="anchor" id="leaf_measurements"></a>

## Section 7: Analyze leaf dimensions
Using placed pseudo-landmarks representing leaves that are rotated upwards and scaled in centimeters from `cm_arr`, calculate the following:

- `width`: difference in centimeters between minimum and maximum x values in an oriented leaf
- `length`: difference in centimeters between minimum and maximum y values in an oriented leaf
- `area`: area of the leaf in centimeters squared
- `solidity`: the ratio of area to convex hull area
- `asymmetry`: the Procrustes distance between the superimposed left and right sides of a leaf outline. Lower values are more symmetric. Higher values are more asymmetric.  

Data is stored in the `mdata` dataframe.

In [ ]:
# lists to store variables
width_list = []
length_list = []
area_list = []
solidity_list = []
asymmetry_list = []

# for each leaf . . .
for lf in range(len(cm_arr)):
    
    # for calculating dimensions, we need non-scaled leaves in centimeters
    curr_lf = cm_arr[lf,:,:] # select current leaf
    
    ############################
    ### CALCULATE DIMENSIONS ###
    ############################
    
    width = np.max(curr_lf[:,0])-np.min(curr_lf[:,0]) # calculate width
    length = np.max(curr_lf[:,1])-np.min(curr_lf[:,1]) # calculate length
    area = poly_area(curr_lf[:,0],curr_lf[:,1]) # calcualte area
    
    ##########################
    ### CALCULATE SOLIDITY ###
    ##########################
    
    hull = ConvexHull(curr_lf) # calculate convex hull of current leaf
    vertices = hull.vertices # isolate vertex indices of convex hull
    convex_area = poly_area(curr_lf[vertices,0], curr_lf[vertices,1]) # calculate convex area
    solidity = area / convex_area # calculate solidity
    
    ##########################
    ### CALCULATE SYMMETRY ###
    ##########################
    
    left_side = curr_lf[:(res-1)+1,] # isolate left side of leaf
    right_side = curr_lf[(res-1):,] # isolate right side of leaf
    right_side = right_side[::-1] # reverse the right side to align indices with left

    # calculate procrustes distance between left and right side of leaf
    s1, s2, distance = procrustes(left_side, right_side) 
    
    # store data in lists
    width_list.append(width)
    length_list.append(length)
    area_list.append(area)
    solidity_list.append(solidity)
    asymmetry_list.append(distance)
    
# add data to the mdata dataframe
mdata["width"] = width_list
mdata["length"] = length_list
mdata["area"] = area_list
mdata["solidity"] = solidity_list
mdata["asymmetry"] = asymmetry_list

#NOTE: run procurstes 1st 
circularity_vals = []
aspect_ratio_vals = []

# for each Procrustes-adjusted shape
for i in range(np.shape(proc_arr)[0]):
    
    # select current shape
    curr_shape = proc_arr[i] 
    
    # calculate circularity and append to list
    circularity_vals.append(Circularity(curr_shape))
    
    # calculate length, width, and aspect ratio
    length = np.max(curr_shape[:,1]) - np.min(curr_shape[:,1])
    width = np.max(curr_shape[:,0]) - np.min(curr_shape[:,0])
    aspect_ratio_vals.append(width/length)
    
mdata['circ'] = circularity_vals
mdata['ar'] = aspect_ratio_vals

mdata.to_csv('metadata_v2.csv', header=True)

[Back to top](#page_top)

<a class="anchor" id="pairplot"></a>

## Section 8: Explore data - Pairplot of all measured traits

In [ ]:
sns.pairplot(mdata,
             x_vars=["length", "width", "area","circ", "ar"],
             y_vars=["length", "width", "area","circ", "ar"],
             hue="dataset",
             plot_kws={"s": 100, "alpha":0.5, "lw":0}, 
            palette = custom_palette)

plt.savefig('pair_plot_dataset_061125.png', dpi= 600, bbox_inches='tight')

# Color palette <a class="anchor" id="color_palette"></a>

In [ ]:
custom_palette = ['#602FF5', '#2FF5AD', '#F5532F', '#F5E32F', '#A06254', 
                 '#5A5175', '#517568', '#757251']

<a class="anchor" id="morphospace"></a>

## Section 9: Plotting PCA morphospace

In [ ]:
plot_length= 20 # plot length in inches
plot_width= 20 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "dataset" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "darkgray" # color of inverse eigenleaf
lf_alpha = 1 # alpha of inverse eigenleaf
pt_size = 100 # size of data points
pt_linewidth = 0 # lw of data points, set to 0 for no edges
pt_alpha = 0.8 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
#ax_tick_fs = 20 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
        
sns.set_style("white", {"axes.grid": False})  
#xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
#ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
plt.savefig('dataset_morphospace_extended_darkgray.png', dpi= 300, bbox_inches='tight')

### Morphospace PCA

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "dataset" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.5 # alpha of inverse eigenleaf
pt_size = 25 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.50 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.8 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.scatterplot(data=mdata, x="PC1", y="PC2", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_palette)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"

plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('combined_expanded_dataset_061125.png', dpi= 300, bbox_inches='tight')

### PCA morphospace colored by circularity

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 1 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.scatterplot(data=mdata, x="PC1", y="PC2", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('circ_dataset_061125_customcmap.png', dpi= 300, bbox_inches='tight')

## custom continuous colormap <a class="anchor" id="custom_color"></a>

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
custom_colors = ["#9773F5", "#73A2F5", "#F5DA73", "#757161", "#676175", "#616975"]
custom_colors2 = ["#F299DF", "#DBC7F2", "#E2F3C7", "#5C7337", "#733766", "#583D78"]

In [ ]:
custom_cmap = LinearSegmentedColormap.from_list("custom_gradient", custom_colors)
custom_cmap                                           

In [ ]:
custom_cmap2 = LinearSegmentedColormap.from_list("custom_gradient", custom_colors2)
custom_cmap2                                           

[Back to top](#page_top)

<a class="anchor" id="plot_leaves"></a>

## Section 10: Plotting real leaves

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(cm_arr),50) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(5,10,plot_num) # subplot number
    
    plt.plot(cm_arr[i,:,0], cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(cm_arr[i,:,0])-0.1,min(cm_arr[i,:,0])-0.1],
            [cm_arr[i,0,1], cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(cm_arr[i,:,0], cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(cm_arr[i,0,0], cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(cm_arr[i,res-1,0], cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    plt.title(mdata["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('leaves_pseudolandmarks_52.png', dpi= 300, bbox_inches='tight')

### Plot leaves from individual datasets

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(apple_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(apple_cm_arr[i,:,0], apple_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(apple_cm_arr[i,:,0])-0.1,min(apple_cm_arr[i,:,0])-0.1],
            [apple_cm_arr[i,0,1], apple_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(apple_cm_arr[i,:,0], apple_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(apple_cm_arr[i,0,0], apple_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(apple_cm_arr[i,res-1,0], apple_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title(apple_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_apple_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(arabidopsis_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(arabidopsis_cm_arr[i,:,0], arabidopsis_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(arabidopsis_cm_arr[i,:,0])-0.1,min(arabidopsis_cm_arr[i,:,0])-0.1],
            [arabidopsis_cm_arr[i,0,1], arabidopsis_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(arabidopsis_cm_arr[i,:,0], arabidopsis_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(arabidopsis_cm_arr[i,0,0], arabidopsis_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(arabidopsis_cm_arr[i,res-1,0], arabidopsis_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title(arabidopsis_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_arabidopsis.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(cannibas_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(cannibas_cm_arr[i,:,0], cannibas_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(cannibas_cm_arr[i,:,0])-0.1,min(cannibas_cm_arr[i,:,0])-0.1],
            [cannibas_cm_arr[i,0,1], cannibas_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(cannibas_cm_arr[i,:,0], cannibas_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(cannibas_cm_arr[i,0,0], cannibas_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(cannibas_cm_arr[i,res-1,0], cannibas_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title(cannibas_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_cannibas_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(capsella_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(capsella_cm_arr[i,:,0], capsella_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(capsella_cm_arr[i,:,0])-0.1,min(capsella_cm_arr[i,:,0])-0.1],
            [capsella_cm_arr[i,0,1], capsella_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(capsella_cm_arr[i,:,0], capsella_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(capsella_cm_arr[i,0,0], capsella_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(capsella_cm_arr[i,res-1,0], capsella_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title(capsella_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_capsella_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(cocoa_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(cocoa_cm_arr[i,:,0], cocoa_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(cocoa_cm_arr[i,:,0])-0.1,min(cocoa_cm_arr[i,:,0])-0.1],
            [cocoa_cm_arr[i,0,1], cocoa_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(cocoa_cm_arr[i,:,0], cocoa_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(cocoa_cm_arr[i,0,0], cocoa_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(cocoa_cm_arr[i,res-1,0], cocoa_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title(cocoa_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_coca_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(lobeliacm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(lobeliacm_arr[i,:,0], lobeliacm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(lobeliacm_arr[i,:,0])-0.1,min(lobeliacm_arr[i,:,0])-0.1],
            [lobeliacm_arr[i,0,1], lobeliacm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(lobeliacm_arr[i,:,0], lobeliacm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(lobeliacm_arr[i,0,0], lobeliacm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(lobeliacm_arr[i,res-1,0], lobeliacm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title( lobelia_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_lobelia_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(oak_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(oak_cm_arr[i,:,0], oak_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(oak_cm_arr[i,:,0])-0.1,min(oak_cm_arr[i,:,0])-0.1],
            [oak_cm_arr[i,0,1], oak_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(oak_cm_arr[i,:,0], oak_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(oak_cm_arr[i,0,0], oak_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(oak_cm_arr[i,res-1,0], oak_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title( oak_data["dataset"][i], fontsize=13)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_oak_v2.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot random leaves and check that it is working

plt.figure(figsize=(10,10)) # set figure size

rand_indices = np.random.randint(0,len(sugar_cm_arr),10) # generate random indices of leaves

plot_num = 1 # plot counter number

for i in rand_indices:
    
    plt.subplot(1,10,plot_num) # subplot number
    
    plt.plot(sugar_cm_arr[i,:,0], sugar_cm_arr[i,:,1], c="k", lw=0.1) # outline
    plt.plot([min(sugar_cm_arr[i,:,0])-0.1,min(sugar_cm_arr[i,:,0])-0.1],
            [sugar_cm_arr[i,0,1], sugar_cm_arr[i,0,1]+1], c="k", lw=0.5) # cm scale
    plt.scatter(sugar_cm_arr[i,:,0], sugar_cm_arr[i,:,1], c="k", s=0.1) # points
    plt.scatter(sugar_cm_arr[i,0,0], sugar_cm_arr[i,0,1], c = "blue", s=20) # base
    plt.scatter(sugar_cm_arr[i,res-1,0], sugar_cm_arr[i,res-1,1], c= "orange", s=20) # tip
    
    #plt.title( sugar_beets_data["dataset"][i], fontsize=12)
    
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
    plot_num += 1
    
#plt.suptitle(str(res*2-1) + " pseudo-landmarks")
plt.tight_layout()
plt.savefig('pseudolandmarks_sugarbeets_v2.png', dpi= 300, bbox_inches='tight')

[Back to top](#page_top)

<a class="anchor" id="split_datasets"></a>

## Section 11: Split datasets for further PCA morphospace plotting 

Here, you can modify the code to color the PCA morphospace by dataset, circularity, width, area, genotype, population, or any other dimension included in the mdata csv. Below are examples that color the PCA morphospace by circularity. 

In [ ]:
oak_df = mdata[mdata["dataset"] == "Quercus_spp"]
apple_df = mdata[mdata["dataset"] == "Malus_spp"]
lobelia_df = mdata[mdata["dataset"] == "Lobelia_spp"]
sugar_df = mdata[mdata["dataset"] == "B_vulgaris"]
capsella_df = mdata[mdata["dataset"] == "C_bursa_pastoris"]
cannabis_df = mdata[mdata["dataset"] == "C_sativa"]
arabidopsis_df = mdata[mdata["dataset"] == "A_thaliana"]
coca_df = mdata[mdata["dataset"] == "E_coca"]

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.set_style("whitegrid")
sns.scatterplot(data=oak_df, x="PC1", y="PC2", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('oak_circ_061125_customcmap.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 100 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.set_style("whitegrid")
sns.scatterplot(data=oak_df, x="PC1", y="PC2", hue=hue, marker = "*", s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('apple_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 100 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.set_style("whitegrid")
sns.scatterplot(data=apple_df, x="PC1", y="PC2", hue=hue, marker = "X", s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('lobelia_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.set_style("whitegrid")
sns.scatterplot(data=sugar_df, x="PC1", y="PC2", hue=hue, marker = "P", s=pt_size, linewidth=pt_linewidth, alpha=pt_alpha,
               palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('sugar_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.scatterplot(data=capsella_df, x="PC1", y="PC2", marker = "s", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('capsella_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.scatterplot(data=cannabis_df, x="PC1", y="PC2", marker = "D", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('cannabis_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 15 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.scatterplot(data=arabidopsis_df, x="PC1", y="PC2", marker = "p", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('arabidopsis_circ_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_length= 10 # plot length in inches
plot_width= 10 # plot length in inches
numPC1 = 40 # set number of PC1 intervals
numPC2 = 20 # set number of PC2 intervals
hue = "circ" # select the factor to color by
s = 0.06 # set the scale of the eigenleaves
lf_col = "lightgray" # color of inverse eigenleaf
lf_alpha = 0.9 # alpha of inverse eigenleaf
pt_size = 50 # size of data points
pt_linewidth = 1 # lw of data points, set to 0 for no edges
pt_alpha = 0.80 # alpha of the data points
ax_label_fs = 25 # font size of the x and y axis titles
ax_tick_fs = 16 # font size of the axis ticks
face_col = "white" # color of the plot background
grid_alpha = 0.5 # set the alpha of the grid
title = "Procrustean morphospace colored by species" # set title

plt.figure(figsize=(plot_length, plot_width))

PC1_vals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]), numPC1 ) # create PC intervals
PC2_vals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), numPC2 )

for i in PC1_vals: # for each PC1 interval
    for j in PC2_vals: # for each PC2 interval
        
        pc1_val = i # select the current PC1 val
        pc2_val = j # select the current PC2 val

        # calculate the inverse eigenleaf
        inv_leaf = pca.inverse_transform(np.array([pc1_val,pc2_val]))
        inv_x = inv_leaf[0::2] # select just inverse x vals
        inv_y = inv_leaf[1::2] # select just inverse y vals
        
        # plot the inverse eigenleaf
        plt.fill(inv_x*s+pc1_val, inv_y*s+pc2_val, c=lf_col, alpha=lf_alpha)
   
# plot the data on top of the morphospace
sns.set_style("whitegrid")
sns.scatterplot(data=coca_df, x="PC1", y="PC2", hue=hue, s=pt_size, linewidth=pt_linewidth, 
                alpha=pt_alpha, palette = custom_cmap2)

plt.legend(bbox_to_anchor=(1.00, 1.02), prop={'size': 8.9})
xlab = "PC1, " + str(round(pca.explained_variance_ratio_[0]*100,1)) + "%"
ylab = "PC2, " + str(round(pca.explained_variance_ratio_[1]*100,1)) + "%"
#plt.text(0.3, 0.28, "rho = 0.67736", horizontalalignment='left', size='medium', color='black')
plt.xlabel(xlab, fontsize=ax_label_fs)
plt.ylabel(ylab, fontsize=ax_label_fs)
plt.xticks(fontsize=ax_tick_fs)
plt.yticks(fontsize=ax_tick_fs)
plt.gca().set_aspect("equal")
plt.gca().set_facecolor(face_col)
plt.grid(alpha=grid_alpha)
plt.gca().set_axisbelow(True)


plt.savefig('coca_circ_061325.png', dpi= 300, bbox_inches='tight')

[Back to top](#page_top)

<a class="anchor" id="LDA"></a>

## Section 12: Linear Discriminant Analysis

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
##################################################
### PERFORM LINEAR DISCRIMINANT ANALYSIS (LDA) ###
##################################################

# create a df for LDA by species
species_df = pd.DataFrame(data=flat_arr[:,:])

# add the genotype labels
species_df["dataset"] = mdata["dataset"]

# create input and output variables
X = species_df.iloc[:,0:((res*2)-1)*2]
y = species_df["dataset"]

# fit the LDA model
species_model = LinearDiscriminantAnalysis()
species_model.fit(X,y)

# retrieve LDA scalings and coefficients
species_scalings = species_model.scalings_
species_coefs = species_model.coef_

# perform prediction
species_prediction= species_model.predict(X)
comparison_result = [X == y for X, y in zip(y, species_prediction)]

# print out number of correctly predicted results
count_true_pl = comparison_result.count(True)
count_false_pl = comparison_result.count(False)
print("The number of falsely predicted:", count_false_pl)
print("The number of correctly predicted:", count_true_pl)
print("Out of " + str(len(comparison_result)) + " total samples" )

# Create confusion matrix
true_species_values = mdata["dataset"]
predicted_species_values = species_prediction

classes = sorted(mdata["dataset"].unique())

cm_species = confusion_matrix(true_species_values, predicted_species_values, labels=classes)


## Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

#evaluate model
scores = cross_val_score(species_model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print("model performed a mean accuracy of ", np.mean(scores))   

# First subplot: confusion matrix
sns.set(font_scale=1.4)
ax1 = sns.heatmap(cm_species, annot=True, annot_kws={"fontsize":8.2}, fmt="d", cmap="viridis", square=True, cbar=False, xticklabels=classes, yticklabels=classes)
ax1 = plt.title("Confusion matrix")
ax1 = plt.xlabel("Predicted class", fontsize=15)
ax1 = plt.ylabel("Actual class", fontsize=15)

sns.move_legend(ax2, "upper left", bbox_to_anchor=(1, 1), fontsize = 8.5)

plt.savefig('lda_dataset_confusion_total_061325.png', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot of linear discriminant scores by genotype

data_plot = species_model.fit(X, y).transform(X)
species_plot_df = pd.DataFrame(data=data_plot[:,:])
species_plot_df["dataset"] = mdata["dataset"]

species_plot_df = species_plot_df.rename(columns={0:'LD1', 1:'LD2', 2:"LD3", 3:"LD4", 4:"LD5", 5:"LD6"})

fig, (ax4, ax5, ax6) = plt.subplots(ncols=3, sharey=True, figsize=(10,5))

sns.scatterplot(data=species_plot_df, x="LD1", y="LD2", hue="dataset", s=10, ax=ax4, palette = custom_palette)
ax4.tick_params(axis='both', labelsize=15)
ax4.set_xlabel("LD1", fontsize=15)
ax4.set_ylabel("LD2", fontsize=15)
ax4.get_legend().remove()
sns.scatterplot(data=species_plot_df, x="LD3", y="LD4", hue="dataset", s=10, ax=ax5, palette = custom_palette)
ax5.tick_params(axis='both', labelsize=15)
ax5.set_xlabel("LD3", fontsize=15)
ax5.set_ylabel("LD4", fontsize=15)
ax5.get_legend().remove()
sns.scatterplot(data=species_plot_df, x="LD5", y="LD6", hue="dataset", s=10, ax=ax6, palette = custom_palette)
ax6.tick_params(axis='both', labelsize=15)
ax6.set_xlabel("LD5",fontsize=15)
ax6.set_ylabel("LD6", fontsize=15)
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), fontsize = 8.5)


plt.tight_layout()
plt.savefig('lda_panels_061125.png', dpi= 300, bbox_inches='tight')

[Back to top](#page_top)

<a class="anchor" id="mean_leaf"></a>

## Section 14: Plotting the mean leaf for the entire dataset and each species dataset

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "dodgerblue")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('mean_leaf_v2.png', dpi= 300, bbox_inches='tight')

### individual mean shapes

In [ ]:
lobelia_mean = gpa_mean(lobeliacm_arr, landmark_num, dim_num)
capsella_mean = gpa_mean(capsella_cm_arr, landmark_num, dim_num)
arabidopsis_mean = gpa_mean(arabidopsis_cm_arr, landmark_num, dim_num)
oak_mean = gpa_mean(oak_cm_arr, landmark_num, dim_num)
cannabis_mean = gpa_mean(cannibas_cm_arr, landmark_num, dim_num)
coca_mean = gpa_mean(cocoa_cm_arr, landmark_num, dim_num)
apple_mean = gpa_mean(apple_cm_arr, landmark_num, dim_num)
sugar_mean = gpa_mean(sugar_cm_arr, landmark_num, dim_num)

In [ ]:
plt.plot(lobelia_mean[:,0], lobelia_mean[:,1], c = "#F5E32F")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('lobelia_mean_leaf_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(lobelia_mean[:,0], lobelia_mean[:,1], "#F5E32F")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('lobelia_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(capsella_mean[:,0], capsella_mean[:,1], "#602FF5")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('capsella_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(arabidopsis_mean[:,0], arabidopsis_mean[:,1], "#2FF5AD")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('arabidopsis_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(oak_mean[:,0], oak_mean[:,1], "#F5532F")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('oak_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(cannabis_mean[:,0], cannabis_mean[:,1], "#A06254")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('cannabis_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(sugar_mean[:,0], sugar_mean[:,1], "#757251")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('sugar_mean_leaf_v2\\overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(apple_mean[:,0], apple_mean[:,1], "#517568")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('apple_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "black", lw = 3)
plt.fill(coca_mean[:,0], coca_mean[:,1], "#5A5175")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('cocoa_mean_leaf_overlay_061125.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(arabidopsis_mean[:,0], arabidopsis_mean[:,1], c = "orange", lw = 3)
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('arabidopsis_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(cannabis_mean[:,0], cannabis_mean[:,1], c = "purple")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('cannabis_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(oak_mean[:,0], oak_mean[:,1], c = "green")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('oak_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(capsella_mean[:,0], capsella_mean[:,1], c = "darkblue")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('capsella_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(coca_mean[:,0], coca_mean[:,1], c = "brown")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('coca_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(apple_mean[:,0], apple_mean[:,1], c = "pink")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('apple_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plt.plot(sugar_mean[:,0], sugar_mean[:,1], c = "gray")
plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('sugar_mean_leaf.png', dpi= 300, bbox_inches='tight')

In [ ]:
plot_col = "k" # set plot color
a = 0.01 # set alpha
for i in range(np.shape(proc_arr)[0]):
    
    curr_leaf = proc_arr[i,:,:]
    
    plt.plot(curr_leaf[:,0], curr_leaf[:,1], c=plot_col, alpha=a) # plot current leaf with color and alpha
    plt.gca().set_aspect("equal")
    
plt.plot(mean_shape[:,0], mean_shape[:,1], c = "dodgerblue") # plot the mean leaf

plt.gca().set_aspect("equal")
plt.axis("off")

plt.savefig('mean_leaves_overlay_061125.png', dpi= 300, bbox_inches='tight')

[Back to top](#page_top)